In [125]:
import numpy as np
print(f'NumPy version: {np.__version__}')

import pandas as pd
print(f'Pandas version: {pd.__version__}')

import sklearn as sk
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from collections import Counter
print(f'Sklearn version: {sk.__version__}')

#NumPy version: 1.20.1
#Pandas version: 1.2.3
#Sklearn version: 0.24.1

NumPy version: 1.20.1
Pandas version: 1.2.3
Sklearn version: 0.24.1


## Laborator 12

Folosind un set de date - de exemplu de la https://archive.ics.uci.edu/ml/datasets.php?format=&task=&att=&area=&numAtt=&numIns=&type=text&sort=taskDown&view=table - sa se rezolve o problema de clasificare sau regresie, plecand de la intrari de tip text.

Se poate opta pentru codificare BOW, n-grams, word2vec sau altele adecvate. Modelele de predictie pot fi din biblioteca scikit-learn. Puteti folosi pentru preprocesare biblioteca [NLTK](https://www.nltk.org).

Pentru clasificare se va optimiza scorul F1; se vor raporta scorurile F1 si acuratetea. Pentru regresie se va optimiza scorul mean squared error; se vor raporta scorurile MSE, mean absolute error, r2.

Exemple:
1. [Clasificare de SMS-uri](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)
1. [Sentence Classification Data Set](https://archive.ics.uci.edu/ml/datasets/Sentence+Classification#)
1. [Sentiment Labelled Sentences Data Set](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)
1. [Victorian Era Authorship Attribution Data Set](https://archive.ics.uci.edu/ml/datasets/Victorian+Era+Authorship+Attribution)
1. [Amazon Commerce reviews set Data Set](https://archive.ics.uci.edu/ml/datasets/Amazon+Commerce+reviews+set)
1. [Farm Ads Data Set](https://archive.ics.uci.edu/ml/datasets/Farm+Ads)


Se vor investiga minim 2 seturi de date si pentru fiecare din ele minim 4 modele de clasificare sau de regresie. Daca setul de date e deja impartit in train si test, se va folosi ca atare - setul de antrenare se va imparti, suplimentar in train + validation; altfel, se va face  5 fold CV. Valorile optime ale hiperparametrilor vor fi alese prin random search sau grid search.

Pentru fiecare set de date:
1. (2 x 0.5 p) Se descrie setul de date, in limba romana (continut, provenienta, problema etc.)
1. (2 x 1 p) Se face analiza exploratorie, folosind cod Python: distributia claselor sau a valorilor continue de iesire - numeric si grafic, statistici asupra textelor (de exemplu: lungime minima/medie/maxima; cele mai frecvente k cuvinte; clustering etc.). Se va explica fiecare pas si ce se urmareste prin efectuarea lui. Graficele vor avea axele numite (ce se reprezinta, evetual unitate de masura)
1. (2 x 0.5 p) Se face preprocesare de date; se explica in limba romana care sunt metodele de preprocesare folosite, efectul lor pe datele de intrare, ce forma are iesirea obtinuta; se arata efectele pasilor de preprocesare asupra setului de date (noul numar de documente, dinamica vocabularului, trasaturile rezultate etc.) Se pot aduga grafice si tabele la acest pas.
1. (2 x 4 x 0.5 p) Clasificare sau regresie, dupa caz: se face o descriere a modelelor considerate, in limba romana; se descrie modalitatea de cautare a hiperparametrilor; rezultatele obtinute se vor prezenta tabelar, similar cu tema precedenta. 

Se acorda doua puncte din oficiu.

Descrierea modelelor si a pasilor de preprocesare pot fi in sectiuni separate, cu referinte la acestea unde e necesar.

Exemple:
1. [Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
1. [Text Classification with Python and Scikit-Learn](https://stackabuse.com/text-classification-with-python-and-scikit-learn/)
1. [How to Prepare Text Data for Machine Learning with scikit-learn](https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/)

In [65]:
def show_metrics_regression(reg, parameters: dict, x: np.ndarray, y: np.ndarray) -> pd.core.frame.DataFrame:
    """
    Shows the metrics('mean_absolute_error', 'mean_squared_error', 'median_absolute_error') of a regressor.
    
    Args:
        reg: a regressor
        parameters:a dictionary containning the hiperparameters
        x: np.array containning the dataset information
        y: np.array containning the classification of the data
        
    Returns:
        a pandas dataframe with the metrics of a regressor
    """
    gridsrc = GridSearchCV(estimator=reg, 
            param_grid=parameters, cv=3, n_jobs=-1, return_train_score=True)   
    scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error']
    scores1 = cross_validate(gridsrc, x, y, cv=5, scoring=scoring, return_train_score=True)    
    
    df1 = pd.DataFrame(data={'train_neg_mean_absolute_error': scores1['train_neg_mean_absolute_error'],
                            'train_neg_mean_squared_error': scores1['train_neg_mean_squared_error'],                            
                            'test_neg_mean_absolute_error': scores1['test_neg_mean_absolute_error'],
                            'test_neg_mean_squared_error': scores1['test_neg_mean_squared_error']                            
                           })    
    
    result = pd.DataFrame([df1.mean()])
    result.insert(0, 'Model_name', [reg])
    result.insert(1, 'Search_strategy', ['GridSearchCV'])
    return result

In [66]:
parameters_SGDRegressor:dict = {
    'max_iter':[10000],
    'loss': ['squared_loss','epsilon_insensitive','squared_epsilon_insensitive'],
    'penalty' : ['l1', 'l2'],
    'alpha' : [0.001, 0.01, 0.1, 1]
}
    
parameters_RandomForestRegressor:dict = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [10, 50, 100]    
}
    
parameters_Lasso:dict = {
    'alpha':[0.01,0.1,1],
    'tol':[0.0001,0.001,0.01,0.1],
    'selection':['cyclic','random']
}
    
parameters_MLPRegressor:dict = {
    'max_iter':[10000],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.01, 0.1, 1],
    'tol':[0.0001, 0.001, 0.01]
}

In [309]:
def dataset_regression(name:str, x:np.ndarray, y:np.ndarray) -> pd.core.frame.DataFrame:
    """
    This function calls the show_metrics_regression for every regression
    and concatenates the resulting dataframes.
    
    Args:
        name:the name of the dataset
        x:the content of the dataset
        y:the target of the dataset
        
    Returns:
        a dataframe                
    """
    df1 = show_metrics_regression(SGDRegressor(), parameters_SGDRegressor, x, y)
    df2 = show_metrics_regression(RandomForestRegressor(),parameters_RandomForestRegressor, x, y)
    df3 = show_metrics_regression(Lasso(), parameters_Lasso, x, y)
    df4 = show_metrics_regression(MLPRegressor(), parameters_MLPRegressor, x, y)    

    df = pd.concat([df1, df2, df3, df4], axis=0, ignore_index=True)
    df.columns.name = name

    return df

In [312]:
def highlight_max(s) -> list:    
    """
    Highlight the maximum in a dataframe red for maximum and green for minimum.
    
    Returns:
        a list of strings representing the color
    """
    
    max_val:float = s.max()
    min_val:float = s.min()
    return ['background-color: #ff6666' if v==max_val and type(v)
            else 'background-color: #bdfcc2'if v==min_val else '' for v in s]

def finishing(df:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    """
    This functions transforms the metrics of the dataframe from negative to positive
    and highlights the column with the min and max from dataframe
    
    Args:
        df:a dataframe
    
    Returns:
        a dataframe with positive numbers and highlights the maximum and minimum.
    """
    aux = df.columns.name
    df.iloc[:, 2:] = df.iloc[:, 2:].abs()

    df.columns = ['Model_name', 'Search_strategy',
                  'train_mean_absolute_error', 'train_mean_squared_error',
                   'test_mean_absolute_error','test_mean_squared_error']

    df = df.style.apply(highlight_max, subset=df.columns[2:])
    df.columns.name = aux
    return df

In [131]:
def metrics_dataframe(data:np.ndarray) -> None:
    """
    This function calculates the the minimum and maximum lenght of a text.
    Also the mean lenght of the texts.
    
    Parameters:
    df:
        a np.ndarray
    
    Returns:
        None
    """
    texts_lenght:list[int] = [len(text) for text in data]
    print(f'Minimum lenght:{np.min(texts_lenght)}')
    print(f'Maximum lenght:{np.max(texts_lenght)}')
    print(f'Mean lenght:{np.mean(texts_lenght)}')

In [300]:
def frequent_words(data:np.ndarray,k=5) -> None:
    """
    This function calculates the k most frequent words in a text.
    
    Args:
        data:the text
        k:the number of words
    
    Returns:
    None
    """
    words:list = [word for text in data for word in text.split(sep=' ') if len(word)>2]  
    words_to_count:generator = (word for word in words)    
    c:collections.Counter = Counter(words_to_count)
    print(f'Most frequent {k} words:{c.most_common(k)}')

## Data Set Information: SMSSpamCollection

Acest dataset a fost colectat de la surse de cercetare libere de pe internet:

-> O colectie de 425 SMS mesaje spam extrase manual de pe  site-ul Grumbletext. Acesta este un forum din UK in care utilizatorii de smartphones revendica mesajele spam, majoritatea fara sa reporteze mesajul receptionat. Identificarea  mesajelor spam revendicate este un lucru foarte indelungat, implicand scanarea a sute de pagini web. Site-ul Grumbletext este: [Web Link](http://www.grumbletext.co.uk/).
-> Un subset de 3,375 de sms-uri ham alese la intamplare din NUS SMS Corpus (NSC), care este un dataset de 10,0000 mesaje legitime, colectate pentru cercetare la Departamentul de Computer Science de la universitatea nationala din Singapore. Majoritatea mesajelor provin de la singaporezi si studenti din universitate. Mesajele au fost colectate de la voluntari carora li s-au spus ca aceste contributii vor fi facute publice.
-> O lista de 450 de SMS ham colectate din teza Caroline Tag's PhD disponibila la: [Web Link](https://etheses.bham.ac.uk/id/eprint/253/1/Tagg09PhD.pdf).
-> In final, a fost incorporat SMS Spam Corpus v.0.1 Big. Are 1,002 mesaje ham si 322 mesaje spam. Acest dataset a fost folosit in urmatoarele cercetari academice:


[1] GÃ³mez Hidalgo, J.M., Cajigas Bringas, G., Puertas Sanz, E., Carrero GarcÃ­a, F. Continut bazat pe filtrarea spamurilor.2006 ACM Simpozion pe documentul de inginerie (ACM DOCENG'06), Amsterdam, The Netherlands, 10-13, 2006.

[2] Cormack, G. V., GÃ³mez Hidalgo, J. M., and Puertas SÃ¡nz, E. Feature pentru infineria de telefon mobil (SMS) spam filtrare.A 30-a conferinta internationala anuala ACM de cercetare si dezvoltare in recuperare de informatie(ACM SIGIR'07), New York, NY, 871-872, 2007.

[3] Cormack, G. V., GÃ³mez Hidalgo, J. M., and Puertas SÃ¡nz, E. Filtrare spam pentru mesaje scurte. A 16-a conferinta ACM  de administrare de informatie si cunostinte(ACM CIKM'07). Lisbon, Portugal, 313-320, 2007.

In [266]:
smsdata:pd.core.frame.DataFrame = pd.read_csv('data/SMSSpamCollection', sep = '\t', names=["label", "sms"])

In [267]:
smsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   object
 1   sms     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [268]:
smsdata

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [269]:
text_x_sms:np.ndarray = smsdata["sms"].values
text_y_sms:np.ndarray = smsdata["label"].values

In [270]:
print('How are the texts organized: ', type(text_x_sms))
print('How many texts:', len(text_x_sms))
print('A text: ', text_x_sms[300])
print('Associated label:', text_y_sms[300])

How are the texts organized:  <class 'numpy.ndarray'>
How many texts: 5572
A text:  Need a coffee run tomo?Can't believe it's that time of week already
Associated label: ham


In [271]:
metrics_dataframe(text_x_sms)

Minimum lenght:2
Maximum lenght:910
Mean lenght:80.48994974874371


In [301]:
frequent_words(text_x_sms,k=4)

Most frequent 4 words:[('you', 1626), ('the', 1207), ('and', 858), ('for', 650)]


## Codificare BOW

Una dintre cele mai mari probleme cu un text este faptul ca este foarte dezordonat si nestructurat, iar algoritmi de machine learning prefera totul structurat si input-uri bine definite. Prin intermediul tehnici Bag-of-Words putem converti variabilele lungi de text in vectori cu lungime fixa. De asemenea, modelel de machine learning lucreaza cu date numerice mai mult decat date textuale. Mai specific, folosind tehnica bag-of-words (BoW), convertim un text in vectorul de numere echivalent acestuia.

## CountVectorizer
Folosim functia CountVectorizer din libraria Sk-learn pentru a implementa usor modelul BoW precizat anterior.

CountVectorizer produce o cale simpla atat pentru a tokeniza o colectie de documente text cat si a construi un vocabular de cuvinte cunoscute, de asemenea codifica si noi documente folosind acel vocabular.


Primul parametru este max_features, care este setat la 1500, deoarece cand convertesti cuvinte la numere folosing BOW, toate cuvintele unice din toate documentele sunt convertite in feature-uri. Toate documentele pot contine zeci de mii de cuvinte unice, dar cuvintele care au o frecventa joasa de aparitie nu sunt un parametru bun de clasificare a documentelor. Prin urmare setam max_features la 1500, ceea ce inseamna ca vrem sa folosim 1500 cele mai frecvente cuvinte ca si feature-uri pentru antrenarea clasificatorului.


Al doilea parametru este min_df care este setat la 5. Asta corespunde cu numarul minim de documente care ar trebui sa contina aces feature. Deci includem numai acele cuvinte care apar in cel putin 5 documente. Similar pentru max_df, valoare este setata la 0.7, ceea ce corespunde unui procentaj. Aici 0.7 inseamna ca ar trebui sa includa numai acele cuvinte care apar in maximum 70% din toate documentele.


In [302]:
vect:sk.feature_extraction.text.CountVectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)

 In continuare este returnat un vector codat cu lungimea intregului vocabular si un contor pentru fiecare aparitie a unui cuvant in document.

In [277]:
print(vect.fit_transform(text_x_sms))

  (0, 537)	1
  (0, 1350)	1
  (0, 1004)	1
  (0, 331)	1
  (0, 151)	1
  (0, 931)	1
  (0, 647)	1
  (0, 224)	1
  (0, 551)	1
  (0, 1462)	1
  (0, 702)	1
  (0, 279)	1
  (0, 1263)	1
  (0, 547)	1
  (0, 1400)	1
  (1, 920)	1
  (1, 707)	1
  (1, 678)	1
  (1, 1434)	1
  (2, 647)	1
  (2, 496)	1
  (2, 436)	2
  (2, 1450)	1
  (2, 302)	1
  (2, 1292)	3
  :	:
  (5570, 496)	1
  (5570, 1292)	1
  (5570, 581)	1
  (5570, 666)	1
  (5570, 1410)	1
  (5570, 113)	1
  (5570, 737)	1
  (5570, 1170)	1
  (5570, 488)	1
  (5570, 1258)	1
  (5570, 176)	1
  (5570, 885)	1
  (5570, 370)	1
  (5570, 1357)	1
  (5570, 227)	1
  (5570, 1174)	1
  (5570, 423)	1
  (5570, 520)	1
  (5570, 229)	1
  (5570, 557)	1
  (5570, 655)	1
  (5571, 1292)	1
  (5571, 873)	1
  (5571, 667)	2
  (5571, 1321)	1


In [278]:
le = LabelEncoder()

In [280]:
x_sms:np.ndarray = vect.fit_transform(text_x_sms).toarray()
y_sms:np.ndarray = le.fit_transform(text_y_sms)

In [283]:
feature_names:list = vect.get_feature_names()
print('Dimensiunea vocabularului:', len(feature_names))

Dimensiunea vocabularului: 1500


In [284]:
print(feature_names[:20])

['00', '000', '02', '03', '04', '06', '0800', '08000839402', '08000930705', '0870', '08712460324', '08718720201', '10', '100', '1000', '10am', '10p', '11', '11mths', '12']


In [285]:
print(feature_names[-20:])

['yahoo', 'yar', 'yeah', 'year', 'years', 'yep', 'yes', 'yest', 'yesterday', 'yet', 'yijue', 'yo', 'yoga', 'you', 'your', 'yours', 'yourself', 'yr', 'yrs', 'yup']


In [61]:
df_smsdata:pd.core.frame.DataFrame = dataset_regression('SMSSpamCollection_Dataset',x_sms,y_sms)
display(finishing(df_smsdata))

SMSSpamCollection_Dataset,Model_name,Search_strategy,train_mean_absolute_error,train_mean_squared_error,test_mean_absolute_error,test_mean_squared_error
0,SGDRegressor(),GridSearchCV,0.071283,0.017270,0.082515,0.022951
1,RandomForestRegressor(),GridSearchCV,0.023676,0.007019,0.045939,0.023677
2,Lasso(),GridSearchCV,0.152425,0.063174,0.153493,0.064383
3,MLPRegressor(),GridSearchCV,0.051834,0.012641,0.063719,0.019776


In [292]:
f = open("data/farm-ads", "r")
lines:list = f.readlines()

In [293]:
farm_ads_data:pd.core.frame.DataFrame = pd.DataFrame([line.split(' ', 1) for line in lines], columns=['label', 'text'])

In [294]:
farm_ads_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4143 entries, 0 to 4142
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   4143 non-null   object
 1   text    4143 non-null   object
dtypes: object(2)
memory usage: 64.9+ KB


In [295]:
farm_ads_data

,label,text
0,1,ad-jerry ad-bruckheimer ad-chase ad-premier ad...
1,-1,ad-rheumatoid ad-arthritis ad-expert ad-tip ad...
2,-1,ad-rheumatologist ad-anju ad-varghese ad-yonke...
3,-1,ad-siemen ad-water ad-remediation ad-water ad-...
4,-1,ad-symptom ad-muscle ad-weakness ad-genetic ad...
...,...,...
4138,-1,ad-affordable ad-ivf ad-cost ad-efficient ad-i...
4139,1,ad-mozypro ad-business ad-backup ad-affordable...
4140,1,ad-oster ad-line ad-clipper ad-oster ad-factor...
4141,-1,ad-synrevoice ad-schoolconnect ad-trust ad-aut...


In [296]:
text_x_farm_ads:np.ndarray = farm_ads_data['text'].values
text_y_farm_ads:np.ndarray = farm_ads_data['label'].values

In [297]:
print('How are the texts organized: ', type(text_x_farm_ads))
print('How many texts:', len(text_x_farm_ads))
print('A text: ', text_x_farm_ads[256])
print('Associated label:', text_y_farm_ads[256])

How are the texts organized:  <class 'numpy.ndarray'>
How many texts: 4143
A text:  ad-embryo ad-storage ad-experience ad-exceptional ad-service ad-low ad-rate ad-www ad-fairfaxcryobank ad-com title-found header-found found found request url emstorage aspx found server additionally found error encounter try errordocument handle request apache cento mod ssl dav mod auth passthrough frontpage server www fairfaxcryobank com port

Associated label: -1


In [298]:
metrics_dataframe(text_x_farm_ads)

Minimum lenght:32
Maximum lenght:63196
Mean lenght:3220.6830798937967


In [303]:
frequent_words(text_x_farm_ads,k=4)

Most frequent 4 words:[('list', 21454), ('product', 17598), ('com', 11628), ('health', 8432)]


In [304]:
vect:sk.feature_extraction.text.CountVectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
x_farm_ads:np.ndarray = vect.fit_transform(text_x_farm_ads).toarray()
y_farm_ads:np.ndarray = le.fit_transform(text_y_farm_ads)

In [305]:
print(vect.fit_transform(text_x_farm_ads))

  (0, 1356)	1
  (0, 251)	1
  (0, 957)	1
  (0, 549)	1
  (1, 1158)	21
  (1, 78)	22
  (1, 479)	2
  (1, 1365)	5
  (1, 689)	1
  (1, 79)	1
  (1, 1391)	5
  (1, 938)	3
  (1, 1324)	1
  (1, 1410)	5
  (1, 470)	7
  (1, 623)	22
  (1, 621)	3
  (1, 1163)	5
  (1, 15)	3
  (1, 290)	3
  (1, 293)	1
  (1, 1251)	1
  (1, 568)	2
  (1, 1089)	13
  (1, 31)	1
  :	:
  (4142, 1430)	1
  (4142, 614)	1
  (4142, 1197)	1
  (4142, 926)	1
  (4142, 1100)	1
  (4142, 757)	1
  (4142, 212)	1
  (4142, 1164)	1
  (4142, 977)	2
  (4142, 898)	1
  (4142, 413)	1
  (4142, 729)	1
  (4142, 1177)	1
  (4142, 531)	1
  (4142, 438)	1
  (4142, 713)	1
  (4142, 1065)	1
  (4142, 714)	1
  (4142, 1377)	1
  (4142, 1107)	1
  (4142, 813)	1
  (4142, 1347)	1
  (4142, 1195)	1
  (4142, 235)	1
  (4142, 743)	1


In [306]:
feature_names:list = vect.get_feature_names()
print('Dimensiunea vocabularului:', len(feature_names))

Dimensiunea vocabularului: 1500


In [307]:
print(feature_names[:20])

['ability', 'able', 'absolutely', 'abuse', 'acai', 'accept', 'access', 'accessory', 'account', 'accuracy', 'accurate', 'achieve', 'acid', 'acr', 'acreage', 'act', 'action', 'active', 'activity', 'actually']


In [308]:
print(feature_names[-20:])

['word', 'workout', 'world', 'worth', 'wrinkle', 'write', 'written', 'wrong', 'wrote', 'ww', 'www', 'xdl', 'yahoo', 'yoga', 'yoplait', 'york', 'yourself', 'youtube', 'zealand', 'zip']


In [79]:
df_farm_ads_data:pd.core.frame.DataFrame = dataset_regression('farm_ads_dataset', x_farm_ads, y_farm_ads)
display(finishing(df_farm_ads_data))

farm_ads_dataset,Model_name,Search_strategy,train_mean_absolute_error,train_mean_squared_error,test_mean_absolute_error,test_mean_squared_error
0,SGDRegressor(),GridSearchCV,0.453148,11.951657,0.437089,0.382943
1,RandomForestRegressor(),GridSearchCV,0.074739,0.017214,0.163477,0.079648
2,Lasso(),GridSearchCV,0.280764,0.114929,0.302685,0.140455
3,MLPRegressor(),GridSearchCV,0.251333,0.320551,0.360259,0.402743


## Random Forest Regressor
<div style="text-align: justify">
&emsp;&emsp;Pădurile aleatoare sau pădurile de decizie aleatoare sunt o metodă de învățare utilizată pentru clasificare, regresie și alte sarcini care operează prin construirea unei multitudini de arbori de decizie în etapa de antrenare și are ca rezultat clasa care are votul majoritar (pentru clasificare) sau media predicțiilor (pentru regresie) a arborilor individuali. Pădurile aleatoare corectează tendința arborilor de decizie de învățare excesivă a datelor de antrenare (overfit).
    
![random_forest_image1](./images/RandomForest.png)

&emsp;&emsp;Algoritmul de antrenament pentru pădurile aleatoare aplică tehnica generală de agregare bootstrap ("bagging") pentru arborii de învățare. Pentru un set de antrenare cu intrările $X = x_1, ..., x_n$ și ieșirile corespunzătoare $Y = y_1, ..., y_n$ prin această tehnică aplicată de $B$ ori se selectează un subset aleator din setul de date de antrenare și antrenează câte un arbore de decizie pentru acest subset. După antrenare, predicțiile pentru exemplele nevăzute $x'$ se obțin prin calculul mediei predicțiilor a arborilor individuali pentru $x'$.<br>
&emsp;&emsp;&emsp;&emsp;For $b = 1, ..., B$:<br>
       &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1. Sample, with replacement, $n$ training examples from $X$, $Y$; call these $X_b$, $Y_b$.<br>
       &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2. Train a classification or regression tree $f_b$ on $X_b$, $Y_b$.<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$y'=\hat{f} = {\frac {1}{B}}{\sum _{b=1}^{B}f_b(x')}$<br>
&emsp;&emsp;Folosirea metodei de agregare bootstrap conduce la performanțe mai bune deoarece reduce varianța modelului, fără a crește bias-ul. Asta înseamnă că deși predicțiile unui arbore sunt sensibile la zgomotul din subsetul său de antrenare, media predicțiilor arborilor nu este, cât timp arborii sunt independenți (antrenați pe subseturi disjuncte de date).<br>
&emsp;&emsp;Algoritmul pădurilor aleatoare diferă de metoda descrisă prin faptul că în momentul împărțirii candidaților se selectează aleator un subset de atribute ale acestora ("feature bagging"), fiecare arbore având acces la un subset aleator din datele de antrenare. Astfel crește diversitatea pădurii, cea ce conduce la predicții mai robuste.<br>

Printre hiper-parametrii utilizați în algoritmul pădurilor aleatoare sunt:
- numărul de arbori, $B$ (n_estimators) pe care algoritmul îi construiește; în general un număr mai mare de arbori crește performanța și conduce la predicții mai stabile, dar scade viteza de calcul
- numărul maxim de atribute (max_features) care să fie luate în considerare la împărțirea candidaților
- numărul minim de frunze necesar pentru a împărți un nod intern (min_sample_leaf)

https://en.wikipedia.org/wiki/Random_forest<br>
https://medium.com/@williamkoehrsen/random-forest-simple-explanation-377895a60d2d<br>
https://towardsdatascience.com/the-random-forest-algorithm-d457d499ffcd
</div>

## Lasso Regressor
<div style="text-align: justify">
&emsp;&emsp;Regresia "lasou" este o metodă analitică de regresie care efectuează atât selecția variabilelor, cât și regularizarea, pentru a spori acuratețea predicției și interpretabilitatea modelului statistic pe care îl produce. Lasso este un tip de regresie liniară care folosește "contracția" datelor (shrinkage). Aceata presupune reducerea datelor spre un punct central, precum media. Procedura lasso încurajează modele simple, rare (adică modele cu mai puțini parametri). Acest tip particular de regresie este bine adaptat modelelor care prezintă niveluri ridicate de muticoliniaritate sau când doriți să automatizați anumite părți ale selecției modelului, cum ar fi selectarea variabilelor / eliminarea parametrilor.
    
![lasso image](./images/Lasso.png)

&emsp;&emsp;Acronimul "LASSO" reprezintă $L$east $A$bsolute $S$hrinkage and $S$election $O$perator(operator de contracție și selecție absolută minimă).<br>
&emsp;&emsp;Regresia Lasso efectuează regularizarea L1, care adaugă o penalizare egală cu valoarea absolută a magnitudinii coeficienților. Acest tip de regularizare poate duce la modele rare, cu câțiva coeficienți; Unii dintre aceștia devenind zero și putând fi eliminați din model. Penalizările mai mari generează valori ale coeficienților mai aproape de zero, ceea ce este ideal pentru producerea de modele mai simple. Pe de altă parte, regularizarea L2 (folosită, de exemplu, în regresia Ridge) nu are ca rezultat eliminarea coeficienților sau a modelelor rare. Acest lucru face Lasso mult mai ușor de interpretat decât Ridge.<br>
&emsp;&emsp;Soluțiile Lasso sunt probleme de programare patrate, care sunt cel mai bine rezolvate cu software, biblioteci specializate (cum ar fi scikit-learn). Scopul algoritmului este de a minimiza:

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;${\sum _{i=1}^{n}{\left (  y_i-{\sum _{j}{x_{ij}\beta_j}} \right )^2}}+{\alpha}{\sum _{j=1}^{p}{|\beta_j|}}$

&emsp;&emsp;Care este același cu minimizarea sumei de pătrate cu constrângerea ${\sum _{j=1}^{p}{|\beta_j|}}\leqslant s$. Unele valori de $\beta$ sunt reduse la zero, rezultând un model de regresie mai ușor de interpretat.


&emsp;&emsp;Printre hiper-parametrii utilizați în algoritmul Lasso din biblioteca scikit-learn este $alpha$ (puterea regularizării $L1$). $alpha$ este cu alte cuvinte proporția de reducere:
- atunci când $alpha$ = 0, nu se elimină nici un parametru. Estimarea este egală cu cea găsită cu regresia liniară.
- pe măsură ce $alpha$ crește, tot mai mulți coeficienți sunt setați la zero și eliminați (teoretic, atunci când $alpha$ = $\infty$ , toți coeficienții sunt eliminați).
- pe măsură ce $alpha$ crește, bias crește.
- pe măsură ce $alpha$ scade, variance crește.<br>
&emsp;&emsp;Dacă în model este inclus un intercept, acesta este de obicei lăsat neschimbat.

https://www.statisticshowto.datasciencecentral.com/lasso-regression/<br>
https://en.wikipedia.org/wiki/Lasso_(statistics)<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html<br>
https://www.slideshare.net/kaz_yos/visual-explanation-of-ridge-regression-and-lasso<br>
https://www.youtube.com/watch?v=NGf0voTMlcs<br>

## Multi-Layer Perceptron Regressor
&emsp;&emsp;Reţelele neurale multistrat — sau perceptronii multistrat, multilayer perceptrons (MLPs) — sunt folosite pentru probleme de regresie, de clasificare şi de estimare de probabilităţi condiţionate. Instruirea este supervizată. Sunt cea mai populară variantă de reţele neurale artificiale şi fac parte din clasa mai mare a reţelelor cu propagare înainte (feed-forward). <br>

&emsp;&emsp;O reţea multistrat se compune din minim trei straturi:
- strat de intrare ce preia valorile de intrare; nu are rol computaţional, nu este format din neuroni;
- cel puțin un strat ascuns, compus din neuroni;
- strat de ieşire, de asemenea compus din neuroni, produce estimări de valori care sunt apoi comparate cu ieşirile dorite;

![mlp image](./images/MLP.png)

&emsp;&emsp;Un strat ascuns este unul care nu primeşte direct intrări şi nu produce valori de ieşire. Neuronii ascunşi produc trăsături noi pe baza vectorilor de intrare, trăsături care sunt mai apoi necesare reţelei neurale pentru producerea unei estimări. Este posibil ca o reţea să aibă mai mult de un neuron în stratul de ieşire. Se consideră că instruirea e mai eficientă dacă pe lângă valorile de intrare şi pe lângă valorile calculate de un strat de neuroni se mai furnizează o valoare constantă, de regulă +1, înmulţită cu o pondere de bias. Ponderile dintre straturi precum şi aceste ponderi de bias sunt instruibile, adică se vor modifica prin procesul de învăţare.

&emsp;&emsp;Rețelele neurale cu propagare înainte precum perceptronul multistrat realizează în principal două mișcări, una înainte și una inversă. Odată ce arhitectura reţelei e fixată – numărul de straturi ascunse şi numărul de neuroni în fiecare strat precum şi funcţiile de activare – se poate trece la instruirea şi apoi utilizarea ei. Pasul de propagare înainte preia un vector de intrare $x = (x_1, . . . , x_n)^t$ şi produce modificări în starea neuronilor reţelei pornind de la intrare şi acţionând asupra succesiv straturilor $2, . . . , L −1$. Ieşirile din ultimul strat sunt
folosite pentru predicţie – regresie, estimare de probabilitate condiţionată sau clasificare. La antrenarea rețelei, în etapa de propagare înainte, semnalul trece de la stratul de intrare, prin straturile ascunse, iar rezultatul obținut în stratul de ieșire este comparat cu cel adevărat din setul de antrenare; în etapa de propagare înapoi (backpropagation), derivatele parțiale ale funcției de eroare cu respect față de diferitele ponderi și valori de bias sunt retro-propagate prin rețea. Prin efectuarea diferenței dintre predicția rețelei și valuarea așteptată se obține un gradient față de care se actualizează parametrii rețelei, aducând perceptronul multistrat mai aproape de eroarea minimă.

&emsp;&emsp;Fiecare pereche din setul de instruire $(x, d)$  va produce valoare de eroare astfel: se furnizează vectorul $x$ ca intrare în reţea şi se calculează un vector de ieşire $o$, reprezentând estimarea produsă de reţea pentru intrarea
furnizată; se foloseşte o funcţie de cost, sau de eroare, care se doreşte a fi cu atât mai mică cu cât vectorul $o$ e mai apropiat de $d$, şi cu atât mai mare cu cât cei doi vectori sunt mai depărtaţi. În plus, se mai consideră un factor de regularizare care împiedică ponderile să devină prea mari în valoare absolută, caz asociat de regulă cu un comportament instabil al reţelei: variaţii mici ale intrării duc la salturi mari în straturile ascunse şi la ieşire.


&emsp;&emsp; În cazul regresiei, neuronii din stratul de ieșire nu au funcție de activare (sau funcția identitate ca funcție de activare), funcția de cost fiind eroarea medie pătratică, iar ieșirea rețelei constă într-un set de valori continue. 

https://skymind.ai/wiki/multilayer-perceptron <br>
https://github.com/lmsasu/cursuri/blob/master/InteligentaArtificiala/curs/InteligentaArtificiala.pdf

## SGDRegressor
Regresie logistică este o modalitate de a modela un sistem discret folosind un funcția logistică (sau o variantă similară). Adică, un sistem cu ieșire are un număr finit de valori posibile. Vă puteți gândi la asta ca la un fel de clasificare algoritm (deși această descriere poate fi periculoasă, deoarece clasificarea este tehnic diferită de regresie), care mapează un set de intrări la un set finit de ieșiri.

Coborârea cu gradient stochastic(SGD) este o variantă a coborâre în gradient (sau coborârea în gradient a lotului) algoritm de optimizare. În loc să utilizeze simultan toate (sau un „lot” de) date de antrenament (poate fi foarte costisitor de calcul / memorie), folosește o aproximare iterativă pentru găsirea minimului unei funcții în spațiul de intrare N-dimensional.

Coborârea cu gradient stochastic poate fi folosit pentru a construi un regresie logistică model similar cu modul în care poate fi folosit pentru a construi un model de regresie liniară. Modelul în sine este independent de algoritmul de optimizare utilizat pentru antrenarea acestuia. In timp ce coborâre gradient stochastic este frecvent utilizat ca algoritm de antrenament, nu este NUMAI opțiune.


Atat estimarea statistica cat si machine learning i-au in considerare problema minimizarii uneu functii obiectiv care are forma sumei:
${\displaystyle Q(w)={\frac {1}{n}}\sum _{i=1}^{n}Q_{i}(w),}{\displaystyle Q(w)={\frac {1}{n}}\sum _{i=1}^{n}Q_{i}(w),}$
unde parametrul  ${\displaystyle w} {\displaystyle Q(w)}$ Q(w) este estimat.Fiecare suma estimata ${\displaystyle Q_{i}}Q_{i}$ este tipic asociata cu   ${\displaystyle i}$  a i-a observatie in dataset(folosit pt antrenare).

Problema sumei minimizare aprea deasemenea ca risc pentru minimizare empirica./ In acest caz, ${\displaystyle Q_{i}(w)}Q_{i}$ (w) este valoarea functiei de loss la  ${\displaystyle i}-lea              exemplu, si {\displaystyle Q(w)}$ Q(w) este riscul empiric.

In stochastic (or "on-line") gradient descent, the true gradient of {\displaystyle Q(w)}Q(w) is approximated by a gradient at a single example:

${\displaystyle w:=w-\eta \nabla Q_{i}(w).}w:=w-\eta \nabla Q_{i}$(w).
In timp ce algoritmul trece prin setul de antrenare,aplica updateul de mai sus pentru fiecare exemplu de antrenare.Cateva treceri pot fi facute peste setul de antrenare pana cand algoritmul converge.Daca se face asta,data poate sa fie amestecate la fiecare pas pentru a preveni ciclurire.Implementarire tipice pot folosi un ritm de invatare adaptiv pentru ca algoritmul sa convearga.

![SGD image](./images/SGD.png)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html
https://en.wikipedia.org/wiki/Stochastic_gradient_descent
https://ro.ec-europe.org/923532-relationship-logistic-regression-and-stochastic-AJDLNA